<a href="https://colab.research.google.com/github/passion1014/KoAlpaca/blob/first-branch/polyglot_test01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
# -*- coding: utf-8 -*-


In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
# ----------------------------------------------------------
# import 선언
# ----------------------------------------------------------
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
#from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [ ]:
# ----------------------------------------------------------
# 변수 선언
# ----------------------------------------------------------
model_name = 'EleutherAI/polyglot-ko-1.3b'
model_path = f'/content/drive/MyDrive/models/huggingface/{model_name}.pt'


In [ ]:
# ----------------------------------------------------------
# HuggingFace에서 모델을 가져와 구글 드라이브에 저장한다.
# ----------------------------------------------------------

from google.colab import drive

# 11.9G 모델 사이즈
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to( "cuda" )    

# 모델 저장
torch.save(model.state_dict(), "/content/drive/MyDrive/models/huggingface/"+model_name+".pt")

# 구글 드라이브 마운트
drive.mount('/content/drive')

# 모델 파일을 구글 드라이브에 복사
!cp {model_name + ".pt"} "/content/drive/MyDrive/models/huggingface/"

In [ ]:
# ----------------------------------------------------------
# 구글에 있는 모델을 로딩한다
# ----------------------------------------------------------
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig

# Load the configuration from the model
config = AutoConfig.from_pretrained(model_name)

# Create the model without weights
model = AutoModelForCausalLM.from_config(config)

# Load the weights into the model
state_dict = torch.load(model_path)
model.load_state_dict(state_dict)

# Move the model to the GPU
model.to('cuda')


In [ ]:

# 테스트



# input_text를 토큰 -> pytorch tensor로 변환
input_text = "How about today"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to('cuda') # "pt" -> pytorch tensor로 변환


# 입력 텐서와 동일한 크기의 Attention mask 생성
# (torch.ones_like() 함수는 모든 요소를 1로 채우는 텐서를 생성)
attention_mask = torch.ones_like(input_ids).to('cuda')


# 모델의 pad token id를 모델의 eos token id로 설정
# (이렇게 하면 모델이 패딩 토큰을 인식하고 적절하게 처리할 수 있다)
model.config.pad_token_id = model.config.eos_token_id


# generate() 메서드를 사용하여 입력 텐서와 attention mask를 기반으로 모델에서 텍스트를 생성
# max_length 인자는 생성할 최대 길이를 지정
output = model.generate(input_ids, attention_mask=attention_mask, max_length=100)


# 생성된 출력 텐서를 CPU 상에서 numpy array로 변환하고, 첫 번째 요소를 선택합니다.
# 선택한 출력을 토크나이저를 사용하여 디코딩하여 생성된 텍스트를 얻습니다.
# skip_special_tokens=True로 설정하여 특수 토큰을 제외한 텍스트만 얻습니다.
generated_text = tokenizer.decode(np.array(output.cpu())[0], skip_special_tokens=True)


# 생성된 텍스트를 출력합니다.
print(generated_text)

In [ ]:
from datasets import load_dataset

ds = load_dataset("beomi/KoAlpaca-v1.1a", split="train")
ds


In [ ]:
len(ds)

# 새 섹션